In [111]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import DoubleType, LongType,IntegerType, DateType, DecimalType



In [2]:
spark = SparkSession.builder.appName("ArticleTransaction").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/06/10 00:29:07 WARN Utils: Your hostname, Abhijit resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/10 00:29:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/10 00:29:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [82]:
# Read CSV files with inferred schema and header
articles_df = spark.read.csv("../article.csv", header=True, inferSchema=False)
transactions_df = spark.read.csv("../transactions.csv", header=True, inferSchema=False)

## Article DF inital exploration

In [ ]:
print("Articles DataFrame:")
articles_df.show(5)
print("Articles DataFrame Schema:")
articles_df.printSchema()
print("Count of Articles DataFrame:", articles_df.count())
print("Unique Article IDs in Articles DataFrame:", articles_df.select("ARTICLE_ID0MATERIAL").distinct().count())
# 4.1 Check for duplicate keys in ARTICLE
dup_articles = articles_df.groupBy("ARTICLE_ID0MATERIAL").count().filter(col("count") > 1)
print("Duplicate ARTICLE_ID0MATERIAL rows:", dup_articles.count())

print("Is Article ID having null values?", articles_df.filter(col("ARTICLE_ID0MATERIAL").isNull()).count() > 0)
print("Is Article ID having empty values?", articles_df.filter(col("ARTICLE_ID0MATERIAL") == "").count() > 0)
print("Is Article ID having empty null strings?", articles_df.filter(col("ARTICLE_ID0MATERIAL") == "null").count() > 0)
# print("Article df statistics:")
# articles_df.describe().show()

Articles DataFrame:
+-------------------+-------------------------+---------------------------+----------+--------------------+----------------------+--------------------+--------------------------------+-------------------+-------------------+
|ARTICLE_ID0MATERIAL|ARTICLE_COLOR_ID0RT_COLOR|ARTICLE_GROUP_ID0RT_CONFMAT|EAN0EANUPC|   DESCRIPTION0TXTMD|BRAND_NAME/SOL/MDPROD1|PICTURE_PATH0EXT_URL|INITIAL_SEASON_NAME/SOL/FMSSEASO|CURRENT_SEASON_NAME|MATL_TYPE0MATL_TYPE|
+-------------------+-------------------------+---------------------------+----------+--------------------+----------------------+--------------------+--------------------------------+-------------------+-------------------+
| 000000000001282247|                     null|                       null|      null|d3dd68a11f543d039...|                    75|                null|                          202008|             202008|               ZMO3|
| 000000000001282474|                     null|                       null|     

### I can observe ARTICLE_ID0MATERIAL is long. Lot of column values are having "null" string which can be converted to actual null values. There are no duplicate rows in article dataframe. Lot of column can make use of datatype change.

In [109]:
# Replace "null" strings with actual nulls
cleaned_articles_df = articles_df.replace("null", None)

#Type conversion
cleaned_articles_df = (
    cleaned_articles_df
      # numeric identifiers → LongType
      .withColumn("ARTICLE_ID0MATERIAL",   col("ARTICLE_ID0MATERIAL").cast(LongType()))
      .withColumn("ARTICLE_GROUP_ID0RT_CONFMAT", col("ARTICLE_GROUP_ID0RT_CONFMAT").cast(LongType()))
      .withColumn("EAN0EANUPC", col("EAN0EANUPC").cast(LongType()))
      .withColumn("BRAND_NAME/SOL/MDPROD1",     col("BRAND_NAME/SOL/MDPROD1").cast(IntegerType()))
      .withColumn("INITIAL_SEASON_NAME/SOL/FMSSEASO",     col("INITIAL_SEASON_NAME/SOL/FMSSEASO").cast(IntegerType()))
      .withColumn("CURRENT_SEASON_NAME",     col("CURRENT_SEASON_NAME").cast(IntegerType()))

)

cleaned_articles_df.show(5)

+-------------------+-------------------------+---------------------------+----------+--------------------+----------------------+--------------------+--------------------------------+-------------------+-------------------+
|ARTICLE_ID0MATERIAL|ARTICLE_COLOR_ID0RT_COLOR|ARTICLE_GROUP_ID0RT_CONFMAT|EAN0EANUPC|   DESCRIPTION0TXTMD|BRAND_NAME/SOL/MDPROD1|PICTURE_PATH0EXT_URL|INITIAL_SEASON_NAME/SOL/FMSSEASO|CURRENT_SEASON_NAME|MATL_TYPE0MATL_TYPE|
+-------------------+-------------------------+---------------------------+----------+--------------------+----------------------+--------------------+--------------------------------+-------------------+-------------------+
|            1282247|                     NULL|                       NULL|      NULL|d3dd68a11f543d039...|                    75|                NULL|                          202008|             202008|               ZMO3|
|            1282474|                     NULL|                       NULL|      NULL|d6045a9b489ed9

## Transaction DF initial exploration

In [ ]:
print("Transaction DataFrame:")
transactions_df.show(5,truncate=False)
# print("Transaction DataFrame Schema:")
# transactions_df.printSchema()
print("Count of Transaction DataFrame:", transactions_df.count())
print("Unique TRANSACTION_ID in Transaction DataFrame:", transactions_df.select("TRANSACTION_ID/SOL/BONKEY").distinct().count())
# 4.1 Check for duplicate keys in ARTICLE
dup_transaction = transactions_df.groupBy("TRANSACTION_ID/SOL/BONKEY").count().filter(col("count") > 1)
print("Duplicate TRANSACTION_ID/SOL/BONKEY rows:", dup_transaction.count())
dup_transaction = transactions_df.groupBy(["TRANSACTION_ID/SOL/BONKEY","ARTICLE_ID0MATERIAL"]).count().filter(col("count") > 1)
print("Duplicate TRANSACTION_ID/SOL/BONKEY, ARTICLE_ID0MATERIAL rows:", dup_transaction.count())
print("Unique TRANSACTION_ID, Article ID in Transaction DataFrame:", transactions_df.select(["TRANSACTION_ID/SOL/BONKEY", "ARTICLE_ID0MATERIAL"]).distinct().count())

#dup_transaction_1 = cleaned_transactions_df.filter(cleaned_transactions_df['TRANSACTION_ID/SOL/BONKEY'] == '0002021040720240000000117' )
#dup_transaction_1.show( truncate=False)

print("Is Article ID having null values?", transactions_df.filter(col("ARTICLE_ID0MATERIAL").isNull()).count() > 0)
print("Is Article ID having empty values?", transactions_df.filter(col("ARTICLE_ID0MATERIAL") == "").count() > 0)
print("Is Article ID having empty null strings?", transactions_df.filter(col("ARTICLE_ID0MATERIAL") == "null").count() > 0)

print("Is Transaction ID having null values?", transactions_df.filter(col("TRANSACTION_ID/SOL/BONKEY").isNull()).count() > 0)
print("Is Transaction ID having empty values?", transactions_df.filter(col("TRANSACTION_ID/SOL/BONKEY") == "").count() > 0)
print("Is Transaction ID having empty null strings?", transactions_df.filter(col("TRANSACTION_ID/SOL/BONKEY") == "null").count() > 0)

Transaction DataFrame:
+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|TRANSACTION_ID/SOL/BONKEY|TRANSACTION_TIME0RPA_ETS2|SALESORG0SALESORG|TRANSACTION_DATE0CALDAY|DISTR_CHAN0DISTR_CHAN|ARTICLE_ID0MATERIAL|LOCATION_ID0PLANT|TRANSACTION_TYPE0RPA_TTC|ARTICLE_COUNT0RPA_RLQ|SALES_PRICE_AT_CASH_DESK0RPA_SAT|SALES_PRICE_PLANNED/SOL/LOC0086C|VAT0RPA_TAM|
+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|0002021040620240000000061|121315                   |1099             |20210406               |13                   |00000000206204202

### After analysis I found that the data does not have any duplicate rows. It has a composite key (TRANSACTION_ID/SOL/BONKEY and ARTICLE_ID0MATERIAL). The data type of the coulmns needs to be defined explicitly.

In [ ]:
# Replace "null" strings with actual nulls
cleaned_transactions_df = transactions_df.replace("null", None)

#Type conversion
cleaned_transactions_df = (
    cleaned_transactions_df
      #.withColumn("TRANSACTION_ID/SOL/BONKEY",   col("TRANSACTION_ID/SOL/BONKEY").cast(LongType()))   Did not work, so left it as StringType
      .withColumn("SALESORG0SALESORG",     col("SALESORG0SALESORG").cast(IntegerType()))
      .withColumn("TRANSACTION_DATE0CALDAY", to_date(col("TRANSACTION_DATE0CALDAY"), "yyyyMMdd"))
      .withColumn("DISTR_CHAN0DISTR_CHAN",     col("DISTR_CHAN0DISTR_CHAN").cast(IntegerType()))
      .withColumn("ARTICLE_ID0MATERIAL",   col("ARTICLE_ID0MATERIAL").cast(LongType()))
      .withColumn("LOCATION_ID0PLANT",     col("LOCATION_ID0PLANT").cast(IntegerType()))
      .withColumn("TRANSACTION_TYPE0RPA_TTC",     col("TRANSACTION_TYPE0RPA_TTC").cast(IntegerType()))
      .withColumn("ARTICLE_COUNT0RPA_RLQ",     col("ARTICLE_COUNT0RPA_RLQ").cast(IntegerType()))
      .withColumn("SALES_PRICE_AT_CASH_DESK0RPA_SAT", col("SALES_PRICE_AT_CASH_DESK0RPA_SAT").cast(DecimalType(10,2)))
      .withColumn("SALES_PRICE_PLANNED/SOL/LOC0086C", col("SALES_PRICE_PLANNED/SOL/LOC0086C").cast(DecimalType(10,2)))
      .withColumn("VAT0RPA_TAM", col("VAT0RPA_TAM").cast(DecimalType(10,2)))

)

cleaned_transactions_df.show(5, truncate=False)
#cleaned_transactions_df.select(['ARTICLE_COUNT0RPA_RLQ']).distinct().show( truncate=False)

cleaned_transactions_df.printSchema()

+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|TRANSACTION_ID/SOL/BONKEY|TRANSACTION_TIME0RPA_ETS2|SALESORG0SALESORG|TRANSACTION_DATE0CALDAY|DISTR_CHAN0DISTR_CHAN|ARTICLE_ID0MATERIAL|LOCATION_ID0PLANT|TRANSACTION_TYPE0RPA_TTC|ARTICLE_COUNT0RPA_RLQ|SALES_PRICE_AT_CASH_DESK0RPA_SAT|SALES_PRICE_PLANNED/SOL/LOC0086C|VAT0RPA_TAM|
+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|0002021040620240000000061|121315                   |1099             |2021-04-06             |13                   |2062042026         |2024             |10

## Let's join the dataframes

In [152]:

article_transaction_df = cleaned_transactions_df.join(
    cleaned_articles_df,
    on="ARTICLE_ID0MATERIAL",
    how="left" # When used Inner join the row drops to 
)
article_transaction_df.show(5, truncate=False)

# Ensure no duplicates in TARGET 
dup_target = article_transaction_df.groupBy(article_transaction_df.columns).count().filter(col("count") > 1)
print("Duplicate rows in TARGET:", dup_target.count())

print("Count of Transaction DataFrame:", cleaned_transactions_df.count())
print("Count of Joined Article Transaction DataFrame:", article_transaction_df.count())
print("Unique  TRANSACTION_ID, Article ID in Joined Article Transaction DataFrame:", article_transaction_df.select(["TRANSACTION_ID/SOL/BONKEY", "ARTICLE_ID0MATERIAL"]).distinct().count())


article_transaction_df.printSchema()

+-------------------+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+-------------------------+---------------------------+-------------+----------------------------------------+----------------------+----------------------------------------+--------------------------------+-------------------+-------------------+
|ARTICLE_ID0MATERIAL|TRANSACTION_ID/SOL/BONKEY|TRANSACTION_TIME0RPA_ETS2|SALESORG0SALESORG|TRANSACTION_DATE0CALDAY|DISTR_CHAN0DISTR_CHAN|LOCATION_ID0PLANT|TRANSACTION_TYPE0RPA_TTC|ARTICLE_COUNT0RPA_RLQ|SALES_PRICE_AT_CASH_DESK0RPA_SAT|SALES_PRICE_PLANNED/SOL/LOC0086C|VAT0RPA_TAM|ARTICLE_COLOR_ID0RT_COLOR|ARTICLE_GROUP_ID0RT_CONFMAT|EAN0EANUPC   |DESCRIPTION0TXTMD                       |BRAND_NAME/SOL/MDPROD1|PICTURE_PATH0EXT_URL                    |INITIAL_SEASON_NAME/

Duplicate rows in TARGET: 0
Count of Transaction DataFrame: 14779
Count of Joined Article Transaction DataFrame: 14779
Unique  TRANSACTION_ID, Article ID in Joined Article Transaction DataFrame: 14779
root
 |-- ARTICLE_ID0MATERIAL: long (nullable = true)
 |-- TRANSACTION_ID/SOL/BONKEY: string (nullable = true)
 |-- TRANSACTION_TIME0RPA_ETS2: string (nullable = true)
 |-- SALESORG0SALESORG: integer (nullable = true)
 |-- TRANSACTION_DATE0CALDAY: date (nullable = true)
 |-- DISTR_CHAN0DISTR_CHAN: integer (nullable = true)
 |-- LOCATION_ID0PLANT: integer (nullable = true)
 |-- TRANSACTION_TYPE0RPA_TTC: integer (nullable = true)
 |-- ARTICLE_COUNT0RPA_RLQ: integer (nullable = true)
 |-- SALES_PRICE_AT_CASH_DESK0RPA_SAT: decimal(10,2) (nullable = true)
 |-- SALES_PRICE_PLANNED/SOL/LOC0086C: decimal(10,2) (nullable = true)
 |-- VAT0RPA_TAM: decimal(10,2) (nullable = true)
 |-- ARTICLE_COLOR_ID0RT_COLOR: string (nullable = true)
 |-- ARTICLE_GROUP_ID0RT_CONFMAT: long (nullable = true)
 |-- EA

### When we do a inner join, the target df count drops to 7184. Let's find the article ids which are not present in the article dataframe.

In [157]:
# 4.2 Check for missing articles in TRANSACTIONS
missing_articles = cleaned_transactions_df.join(cleaned_articles_df, on='ARTICLE_ID0MATERIAL', how='left_anti')

#Print count of missing articles
print("Count of Transactions with no matching ARTICLE:", missing_articles.count())

print("Total count of transactions:", cleaned_transactions_df.count())
print("Total of missing and non-missing articles in TRANSACTIONS:", 7184 + missing_articles.count())

print("Transactions with no matching ARTICLE:")
missing_articles.show(5)

Count of Transactions with no matching ARTICLE: 7595
Total count of transactions: 14779
Total of missing and non-missing articles in TRANSACTIONS: 14779
Transactions with no matching ARTICLE:
+-------------------+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-----------------+------------------------+---------------------+--------------------------------+--------------------------------+-----------+
|ARTICLE_ID0MATERIAL|TRANSACTION_ID/SOL/BONKEY|TRANSACTION_TIME0RPA_ETS2|SALESORG0SALESORG|TRANSACTION_DATE0CALDAY|DISTR_CHAN0DISTR_CHAN|LOCATION_ID0PLANT|TRANSACTION_TYPE0RPA_TTC|ARTICLE_COUNT0RPA_RLQ|SALES_PRICE_AT_CASH_DESK0RPA_SAT|SALES_PRICE_PLANNED/SOL/LOC0086C|VAT0RPA_TAM|
+-------------------+-------------------------+-------------------------+-----------------+-----------------------+---------------------+-----------------+------------------------+---------------------+--------------------------------+-----------

### Let's write the final table as parquet

In [159]:
article_transaction_df.coalesce(1).write.mode("overwrite").parquet('../article_transaction/')

In [160]:
spark.stop()